In [1]:
import pandas as pd
from gensim.models import FastText
import re
import numpy as np
from libsvm import svmutil
import libsvm
from sklearn.metrics import r2_score, mean_absolute_error
from scipy.stats import kendalltau

In [2]:
model_summary = FastText.load("models-4.1(a)/model_summary.bin")


In [25]:
data = pd.read_csv("data_pos.csv")
head_vec = np.load("part_d_features.npy")

In [26]:
head_vec.shape

(9523, 6)

In [27]:
feat_len = int(0.8*len(head_vec))
train = head_vec[:feat_len]
res_train = list(np.log2(np.array(data["Like Count"][:feat_len].tolist())+1))
test = head_vec[feat_len:]
res_test = list(np.log2(np.array(data["Like Count"][feat_len:].tolist())+1))
print(len(train), len(res_train))

7618 7618


In [28]:
model_svm = svmutil.svm_train(res_train, train,'-s 4 -t 2 -c 2')

In [29]:
y_reg = svmutil.svm_predict(res_test, test, model_svm)

Mean squared error = 0.935617 (regression)
Squared correlation coefficient = 0.00065124 (regression)


In [30]:
r2 = r2_score(res_test,y_reg[0])
r2

-0.05389867746440591

In [31]:
mae = mean_absolute_error(res_test, y_reg[0])
mae

0.731988299805631

In [32]:
kt = kendalltau(res_test, y_reg[0])
kt

KendalltauResult(correlation=0.018765091867413385, pvalue=0.2201078459205713)

In [34]:
y_full = svmutil.svm_predict(res_test+res_train, list(test)+list(train), model_svm)

Mean squared error = 0.827452 (regression)
Squared correlation coefficient = 0.0959599 (regression)


In [35]:
r2 = r2_score(res_test+res_train,y_full[0])
r2

0.08161550170159593

In [36]:
mae = mean_absolute_error(res_test+res_train, y_full[0])
mae

0.6921559615586254

In [37]:
kt = kendalltau(res_test+res_train, y_full[0])
kt

KendalltauResult(correlation=0.2023848371088694, pvalue=1.968954546788391e-192)

In [38]:
y_tr = svmutil.svm_predict(res_train, train, model_svm)

Mean squared error = 0.800404 (regression)
Squared correlation coefficient = 0.141301 (regression)


In [39]:
r2 = r2_score(res_train,y_tr[0])
r2

0.11485608157734051

In [40]:
mae = mean_absolute_error(res_train, y_tr[0])
mae

0.6821952626402025

In [41]:
kt = kendalltau(res_train, y_tr[0])
kt

KendalltauResult(correlation=0.24947237434897407, pvalue=1.99452433311386e-233)

In [43]:
model_svm_reg = svmutil.svm_train(res_train+res_test, list(train)+list(test),'-s 4 -v 10 -t 2 -c 2')

Cross Validation Mean squared error = 0.950648
Cross Validation Squared correlation coefficient = 0.000628807


In [44]:
svmutil.svm_save_model("model_d_reg.bin",model_svm)